In [1]:
# import the neccessary libs
import tensorflow as tf 
import pandas as pd 
import matplotlib.pyplot as plt

2023-01-10 16:33:43.562667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load and read the .csv file
insurance = pd.read_csv('../Night/insurance.csv')
# get the info and description
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [3]:
# insurance.describe()

In [4]:
# one-hot encode the object values  - sex, smoking and region
insurance_onehot = pd.get_dummies(insurance)
insurance_onehot

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [5]:
# # seperate the X and y
# X = insurance_onehot.drop("charges", axis=1)
# y = insurance_onehot["charges"] # have only charges column

In [6]:
# X["age"].plot(kind="hist", figsize=(7,4))
# X["bmi"].plot(kind="hist", figsize=(7,4))
# X["children"].plot(kind="hist", figsize=(7,4))

In [7]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# scale each feature to a given range - 0,1 -> minmax scaler
# identify columns that need to be minimized, and those to be one-hot encoded

# create a column transformer
ct = make_column_transformer(
    (MinMaxScaler(), ["age","bmi","children"]), # normalize values in these columns
    (OneHotEncoder(handle_unknown="ignore"), ["sex", "smoker", "region"])
)

# Create X and y
X = insurance.drop("charges", axis=1)
y = insurance["charges"]

# Build our train and test data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Fit the column transformer to our training data: make our training data coincide with whatever
# we have created
ct.fit(X_train)

# Transform training and test data with normalization (MinMaxScaler) and OneHotEncoder
X_train_norm = ct.transform(X_train)
X_test_norm = ct.transform(X_test)

In [8]:
# what did our data look like previously
X_train.loc[12]

age                23
sex              male
bmi              34.4
children            0
smoker             no
region      southwest
Name: 12, dtype: object

In [9]:
# how does our new look-like
X_train_norm[12]
# its all numeric data

array([0.67391304, 0.37570621, 0.        , 1.        , 0.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        ])

In [10]:
y_train.shape, X_train_norm.shape

((1070,), (1070, 11))

In [11]:
X_train.shape, X_train_norm.shape

((1070, 6), (1070, 11))

* our X_train had 6 columns but after OneHotEncoding (and normalizing), we've ended up with 11 columns on the X_train_norm

`Build a Model`
* the callback wasn't neccessary in this case

In [56]:
# random_seeed
tf.random.set_seed(42)

# create a model
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(10),
        # tf.keras.layers.Dense(10),
        tf.keras.layers.Dense(1)
    ]
)

# compile a model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.25),
    loss = tf.keras.losses.mae, 
    metrics='mae'
)

# instantiate an EarlyStopping callback
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience='15')

# fit the model
model.fit(X_train_norm, y_train, epochs=200, verbose=0) #, callbacks=[callback])

In [57]:
# evaluate the model on the "normalized" data it was trained on
model.evaluate(X_test_norm, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 3168.2529 - mae: 3168.2529


[3168.2529296875, 3168.2529296875]

In [30]:
y_pred = model.predict(X_test_norm)

9/9 [==============================] - 0s 2ms/step


In [58]:
# save my model
model.save("norm-model")

INFO:tensorflow:Assets written to: norm-model/assets


INFO:tensorflow:Assets written to: norm-model/assets
